In [14]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import plotly.graph_objects as go

def plot_calibration(fileName):
    # Load data from Excel
    df = pd.read_excel(fileName, sheet_name="Results")

    # Conductivity (dependent variable)
    y1 = df["Conductivity_A_(mS)"]
    y2 = df["Conductivity_B_(mS)"]
    y3 = df["Conductivity_C_(mS)"]

    # Concentration (independent variable)
    x_from_excel = df["Concentration_(g/L)"]

    # Run number column (assuming it's named "Run" in Excel)
    run_numbers = df["Run"]

    # Concatenate y values and replicate x values
    y_data = np.concatenate([y1, y2, y3])  # Conductivity
    x_data = np.tile(x_from_excel, 3)      # Concentration
    run_data = np.tile(run_numbers, 3)     # Repeat run numbers

    # Split data into two groups based on run number
    mask_group1 = (run_data >= 0) & (run_data <= 4)
    mask_group2 = (run_data >= 5) & (run_data <= 8)

    x_data_group1, y_data_group1 = x_data[mask_group1], y_data[mask_group1]
    x_data_group2, y_data_group2 = x_data[mask_group2], y_data[mask_group2]

    # Fit separate OLS models (forcing zero intercept)
    def fit_model(x, y):
        X = x[:, np.newaxis]
        model = sm.OLS(y, X).fit()
        return model

    model_group1 = fit_model(x_data_group1, y_data_group1)
    model_group2 = fit_model(x_data_group2, y_data_group2)

    # Get confidence intervals
    def get_conf_intervals(model, X):
        pred = model.get_prediction(X)
        pred_summary = pred.summary_frame(alpha=0.05)
        return pred_summary['mean_ci_lower'], pred_summary['mean_ci_upper']

    ci_lower_group1, ci_upper_group1 = get_conf_intervals(model_group1, x_data_group1[:, np.newaxis])
    ci_lower_group2, ci_upper_group2 = get_conf_intervals(model_group2, x_data_group2[:, np.newaxis])


        # Create plotly figure
    fig = go.Figure()

    # --- Plot Group 1 line first (so it's behind) ---
    fig.add_trace(go.Scatter(
        x=x_data_group1,
        y=model_group1.fittedvalues,
        mode='lines',
        name='OLS Model (Runs 0-4)',
        showlegend=False,
        line=dict(color="black", width=2),
        error_y=dict(
            type='data',
            symmetric=False,
            array=ci_upper_group1 - model_group1.fittedvalues,
            arrayminus=model_group1.fittedvalues - ci_lower_group1,
            visible=True,
            color="black"
        )
    ))

    # --- Then plot Group 1 points (on top) ---
    fig.add_trace(go.Scatter(
        x=x_data_group1,
        y=y_data_group1,
        mode="markers",
        marker=dict(symbol="cross", size=10, color="crimson"),  # distinct point color
        name="Runs 0-4"
    ))

    # --- Plot Group 2 line first ---
    fig.add_trace(go.Scatter(
        x=x_data_group2,
        y=model_group2.fittedvalues,
        mode='lines',
        name='OLS Model (Runs 5-8)',
        showlegend=False,
        line=dict(color="black", width=2),
        error_y=dict(
            type='data',
            symmetric=False,
            array=ci_upper_group2 - model_group2.fittedvalues,
            arrayminus=model_group2.fittedvalues - ci_lower_group2,
            visible=True,
            color="black"
        )
    ))

    # --- Then plot Group 2 points (on top) ---
    fig.add_trace(go.Scatter(
        x=x_data_group2,
        y=y_data_group2,
        mode="markers",
        marker=dict(symbol="x", size=10, color="blue"),
        name="Runs 5-8"
    ))


    # # Plot Group 1 (Runs 0-4)
    # fig.add_trace(go.Scatter(x=x_data_group1, y=y_data_group1, mode="markers",
    #                          marker=dict(symbol="x", color="red"), name="Runs 0-4"))

    # fig.add_trace(go.Scatter(
    #     x=x_data_group1,
    #     y=model_group1.fittedvalues,
    #     mode='lines',
    #     name='OLS Model (Runs 0-4)',
    #     line=dict(color="black"),
    #     error_y=dict(
    #         type='data',
    #         symmetric=False,
    #         array=ci_upper_group1 - model_group1.fittedvalues,
    #         arrayminus=model_group1.fittedvalues - ci_lower_group1,
    #         visible=True,
    #         color="black"
    #     )
    # ))

    # # Plot Group 2 (Runs 5-8)
    # fig.add_trace(go.Scatter(x=x_data_group2, y=y_data_group2, mode="markers",
    #                          marker=dict(symbol="circle", color="blue"), name="Runs 5-8"))

    # fig.add_trace(go.Scatter(
    #     x=x_data_group2,
    #     y=model_group2.fittedvalues,
    #     mode='lines',
    #     name='OLS Model (Runs 5-8)',
    #     line=dict(color="blue"),
    #     error_y=dict(
    #         type='data',
    #         symmetric=False,
    #         array=ci_upper_group2 - model_group2.fittedvalues,
    #         arrayminus=model_group2.fittedvalues - ci_lower_group2,
    #         visible=True,
    #         color="blue"
    #     )
    # ))

    # Extract slopes
    # Extract slopes and R² values
    slope_group1 = model_group1.params[0]
    r2_group1 = model_group1.rsquared

    slope_group2 = model_group2.params[0]
    r2_group2 = model_group2.rsquared


    # Add equation annotations
    # fig.add_annotation(
    #     x=x_data_group1.max(),
    #     y=slope_group1 * x_data_group1.max(),
    #     text=f"<b>y = {slope_group1:.4f}x</b> (Runs 0-4)",
    #     font=dict(size=14, color="black"),
    #     xanchor="right",
    #     yanchor="bottom",
    #     bgcolor="white"
    # )

    # fig.add_annotation(
    #     x=x_data_group2.max(),
    #     y=slope_group2 * x_data_group2.max(),
    #     text=f"<b>y = {slope_group2:.4f}x</b> (Runs 5-8)",
    #     font=dict(size=14, color="blue"),
    #     xanchor="right",
    #     yanchor="bottom",
    #     bgcolor="white"
    # )
    # Add equation + R² annotation for Group 1
    fig.add_annotation(
        x=x_data_group1.max(),
        y=slope_group1 * x_data_group1.max(),
        text=f"<b>y = {slope_group1:.4f}x<br>R² = {r2_group1:.4f}",
        font=dict(size=16, color="black"),
        xanchor="right",
        yanchor="bottom",
        bgcolor="white"
    )

    # Add equation + R² annotation for Group 2
    fig.add_annotation(
        x=x_data_group2.max(),
        y=slope_group2 * x_data_group2.max(),
        text=f"<b>y = {slope_group2:.4f}x<br>R² = {r2_group2:.4f}",
        font=dict(size=16, color="blue"),
        xanchor="right",
        yanchor="bottom",
        bgcolor="white"
    )

    # Update layout
    fig.update_layout(
        xaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor="gray",
            title="Concentration (g/L)",  # X-axis
            title_font=dict(size=16),
            tickfont=dict(size=14),
            linewidth=2,
            linecolor="black",
            mirror=True
        ),
        yaxis=dict(
            showgrid=True,
            gridwidth=1,
            gridcolor="gray",
            title="Conductivity (mS)",  # Y-axis
            title_font=dict(size=16),
            tickfont=dict(size=14),
            linewidth=2,
            linecolor="black",
            mirror=True
        ),
        # title=f"Calibration Curve - {fileName}",
        width=800,
        height=600,
        font=dict(family="Arial", size=16),
        plot_bgcolor="white",
        margin=dict(l=60, r=30, t=30, b=60)
    )

    fig.show()

# Run the function for both files
plot_calibration("RO_Week_1_Data.xlsx")
plot_calibration("RO_Week_2_Data.xlsx")
